In [32]:
import pandas as pd
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [27]:
df = pd.read_csv('ml_gw_car_insurance.csv')
df.head()

,Unnamed: 0,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,...,No,No,No,Yes,No,No,No,Yes,0,0.0
1,1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,...,No,No,No,Yes,No,No,No,Yes,0,0.0
2,2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,...,No,No,No,Yes,No,No,No,Yes,0,0.0
3,3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0.0
4,4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0.0


In [16]:
# Number of car models
df.model.nunique()

11

In [28]:
#Transforming Yes and No values into Booleans
df = df.replace({ "No" : False , "Yes" : True })

#Removal of policy_id column
df.drop(columns = {"policy_id", 'Unnamed: 0'}, inplace = True)

df.head()

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,60Nm@3500rpm,...,False,False,False,True,False,False,False,True,0,0.0
1,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,60Nm@3500rpm,...,False,False,False,True,False,False,False,True,0,0.0
2,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,60Nm@3500rpm,...,False,False,False,True,False,False,False,True,0,0.0
3,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,113Nm@4400rpm,...,True,True,True,True,True,True,True,True,2,0.0
4,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,91Nm@4250rpm,...,False,True,True,True,False,True,True,True,2,0.0


In [24]:
print(df.age_of_car.max())
print(df.age_of_car.min())

1.0
0.0


In [25]:
print(df.age_of_policyholder.max())
print(df.age_of_policyholder.min())

1.0
0.288461538461538


In [29]:
df.is_claim.value_counts()/df.shape[0] * 100

# Is_Claim is heavily unbalanced -> use class_weight for logistic regression

0.0    56.160975
1.0     3.838001
Name: is_claim, dtype: float64

In [40]:
df = df[(df.is_claim == 1) | (df.is_claim == 0)]

In [42]:
# linear correlation
corr = df.corr().abs()
corr_claim = corr['is_claim'].sort_values(ascending=False)
print(f'The variables most correlated with claims are as followed: \n{corr_claim}')

The variables most correlated with claims are as followed: 
is_claim                            1.000000
policy_tenure                       0.078747
age_of_car                          0.028172
age_of_policyholder                 0.022435
population_density                  0.017808
is_adjustable_steering              0.013917
cylinder                            0.013434
is_front_fog_lights                 0.011825
is_brake_assist                     0.010893
is_driver_seat_height_adjustable    0.010686
width                               0.009947
is_parking_sensors                  0.008419
is_day_night_rear_view_mirror       0.007989
displacement                        0.007678
is_speed_alert                      0.007307
is_ecw                              0.006637
is_power_door_locks                 0.006637
is_central_locking                  0.006637
length                              0.006495
gross_weight                        0.003894
ncap_rating                         0.00

/var/folders/vb/dl5mj5tx3gb0xhf7g2xbfx980000gn/T/ipykernel_52536/1614279777.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr().abs()


In [ ]:
profile = ProfileReport(df)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]